#  tính chi phí đường đi cho 1 route
route = [0, ..., 0]
N người, đánh số i từ [1,N]. Đón người i ở điểm i, trả ở điểm i+N+M. Nếu đón người i phải có đường đi trực tiếp từ i->i+N+M
M hàng, đánh số j từ [1,M]. Nhận hàng j ở điểm j+N, trả ở điểm j+2N+M. Nếu nhận hàng j thì trong đường đi về sau phải đi qua j+2N+M.
K xe, đánh số k từ [0,K-1]. Khởi hành ở 0, kết thúc sẽ ở 0.
q: danh sách số lượng hàng mỗi loại.
Q: danh sách trọng tải hàng mỗi xe.

In [61]:
import random
import math
import copy

def compute_route_length(route, distance_matrix):
    total_length = 0
    for i in range(len(route) - 1):
        total_length += distance_matrix[route[i]][route[i + 1]]
    return total_length

def compute_max_route_length(solution, distance_matrix):
    return max(compute_route_length(route, distance_matrix) for route in solution)

# khởi tạo trạng thái đầu:
- thỏa mãn về việc xe cần có đường đi trực tiếp từ điểm đón người tới điểm trả người.
- thỏa mãn ràng buộc về trọng tải mỗi xe khi trở hàng

In [62]:
import random
import copy
def initialize_solution(N, M, K,q, Q):
    solution = [[] for _ in range(K)]
    parcels = list(range(N+1, N + M+1))  # Các yêu cầu chở hàng (4,5,6)
    passengers = list(range(1, N + 1))  # Các yêu cầu chở hành khách
    weight = [0 for i in range(K)] # weight là danh sách chứa khối lượng từng xe
    
# Phân bổ yêu cầu hàng vào taxi
    
    for parcel in parcels:
        taxi = random.randint(0, K - 1) # index của xe được chọn
        pickup = parcel # index của hàng được chọn 
        dropoff = parcel + M + N
        
        # Tìm vị trí hợp lệ để chèn
        route = solution[taxi]
        id_pickup = random.randint(0, len(route))  
        
        # nếu khối lượng ít hơn trọng tải thì nhận thêm hàng
        if weight[taxi] + q[pickup-N-1] <= Q[taxi]:
            route.insert(id_pickup, pickup)
            # mỗi khi đón hàng, cập nhật khối lượng hiện tại
            weight[taxi] += q[pickup-N-1]
         
        
      
        # Đảm bảo điểm dropoff được chèn sau điểm pickup
        id_dropoff = random.randint(id_pickup + 1, len(route))
        route.insert(id_dropoff, dropoff)
        weight[taxi] -= q[dropoff-N-M-N-1]
        
    
# Phân bổ yêu cầu chở người

    for passenger in passengers:
        taxi = random.randint(0, K - 1)
        pickup = passenger
        dropoff = passenger + M + N
        
        # Tìm vị trí hợp lệ để chèn
        route = solution[taxi]
        insert_pickup = random.randint(0, len(route))  
        route.insert(insert_pickup, pickup)
        
        # Đảm bảo điểm dropoff được chèn ngay sau điểm pickup
        insert_dropoff = insert_pickup + 1
        route.insert(insert_dropoff, dropoff)
        
    # Thêm điểm depot vào đầu và cuối mỗi route
    for taxi in range(K):
        solution[taxi] = [0] + solution[taxi] + [0]
    
    return solution


# Tạo trạng thái neighbors
- tớ chưa nghĩ ra nên quyết định tạo 10 random solution cho tập neighbors

In [63]:
import random
import copy

def generate_neighbors(solution, N, M, K, q, Q):
    neighbors = []
    for _ in range(10):
        neighbors.append(initialize_solution(N, M, K,q, Q))
    return neighbors    


# hill climbing
- chọn ra phương án tham lam nhất trong tập neighbors
- lưu giá trị tối ưu và so sánh với các neighbors mới được tạo ra
- nếu trong các neighbors mới ko có kết quả tối ưu hơn thì cho ra kết quả

In [64]:
def hill_climbing(N, M, K, q, Q, distance_matrix):
    # 1. Khởi tạo giải pháp ban đầu
    solution = initialize_solution(N, M, K,q, Q)
    best_solution = solution
    best_cost = compute_max_route_length(solution, distance_matrix)

    # 2. Lặp để cải thiện
    iterations = 0
    MAX_ITERATIONS = 1000
    while iterations < MAX_ITERATIONS:
        # Tìm hàng xóm
        neighbors = generate_neighbors(solution, N, M, K, q, Q)

        # Đánh giá hàng xóm
        for neighbor in neighbors:
            cost = compute_max_route_length(neighbor, distance_matrix)
            if cost < best_cost:
                best_solution = neighbor
                best_cost = cost
        
        # Dừng nếu không cải thiện
        if best_solution == solution:
            break
        solution = best_solution
        iterations += 1

    return best_solution, best_cost




# main

In [71]:
# Hàm chính (input và output)
def main():
    # Input
    N, M, K = 3,3,2
    q = [8,4,5]
    Q = [16,16]
    distance_matrix = [
    [0, 8, 7, 9, 6, 5, 11, 6, 11, 12, 12, 12, 13],
    [8, 0, 4, 1, 2, 8, 5, 13, 19, 12, 4, 8, 9],
    [7, 4, 0, 3, 3, 8, 4, 12, 15, 8, 5, 6, 7],
    [9, 1, 3, 0, 3, 9, 4, 14, 19, 11, 3, 7, 8],
    [6, 2, 3, 3, 0, 6, 6, 11, 17, 11, 6, 9, 10],
    [5, 8, 8, 9, 6, 0, 12, 5, 16, 15, 12, 15, 15],
    [11, 5, 4, 4, 6, 12, 0, 16, 18, 7, 4, 3, 4],
    [6, 13, 12, 14, 11, 5, 16, 0, 15, 18, 17, 18, 19],
    [11, 19, 15, 19, 17, 16, 18, 15, 0, 13, 21, 17, 17],
    [12, 12, 8, 11, 11, 15, 7, 18, 13, 0, 11, 5, 4],
    [12, 4, 5, 3, 6, 12, 4, 17, 21, 11, 0, 7, 8],
    [12, 8, 6, 7, 9, 15, 3, 18, 17, 5, 7, 0, 1],
    [13, 9, 7, 8, 10, 15, 4, 19, 17, 4, 8, 1, 0]]

    #for _ in range(2 * N + 2 * M + 1):
        #distance_matrix.append(list(map(int, input().split())))

    # Thực hiện Local Search
    solution, cost = hill_climbing(N, M, K, q, Q, distance_matrix)
    # Output
    print(K)
    for route in solution:
        print(len(route))
        print(" ".join(map(str, route)))

main()

2
8
0 6 12 4 10 2 8 0
8
0 5 3 9 11 1 7 0
